In [2]:
import pandas as pd
import numpy as np
import cv2
import os
import random
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
# we are using efficient net B3 but we can downgrade if it is too computationally expensive
from tensorflow. keras.applications import EfficientNetB3
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import Callback, EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

In [3]:
''' in this notebook we can preprocess images for the CNN. all the images are super nice and prestine, biut we want them to be distorted 
to an extent or "preprocessed" so the CNN can recognize images pf even low quality'''

# the model takes images of size 224,224,3

' in this notebook we can preprocess images for the CNN. all the images are super nice and prestine, biut we want them to be distorted \nto an extent or "preprocessed" so the CNN can recognize images pf even low quality'

In [4]:
# filepaths 
Test_images = '/Users/melissaaprilcastro/FeatherFind/Data/CUB_200_2011/Test'
Train_images = '/Users/melissaaprilcastro/FeatherFind/Data/CUB_200_2011/Train'

In [5]:
# function to collect all image paths from nested subdirectories
def get_image_paths(folder):
    image_paths = []
    # Traverse the directory and subdirectories
    for root, _, files in os.walk(folder):
        for image in files:
            # Check if the file is an image by its extension
            if image.lower().endswith(('.png', '.jpg', '.jpeg', '.gif', '.bmp', '.tiff')):
                image_paths.append(os.path.join(root, image))
    return image_paths  # Return the list of image paths

# function to check image dimensions and size of a certain amt of images
def check_random_image_sizes(folder, num_samples=10):
    # Get all image files from nested subdirectories
    image_paths = get_image_paths(folder)
    
    # check if there are enough images to sample (there are lol)
    if len(image_paths) < num_samples:
        print(f"Only found {len(image_paths)} images, processing all of them.")
        selected_images = image_paths
    else:
        # randomly select a subset of images
        selected_images = random.sample(image_paths, num_samples)
    
    # process each selected image
    for file_path in selected_images:
        try:
            # read the image using OpenCV
            img = cv2.imread(file_path)
            
            # check if image was loaded correctly
            if img is not None:
                # get dimensions: height, width, and number of channels
                height, width, channels = img.shape
                
                # get file size in bytes
                file_size = os.path.getsize(file_path)
                
                print(f"Image: {os.path.basename(file_path)}")
                print(f"  - Dimensions: {width} x {height}")
                print(f"  - Channels: {channels}")
                print(f"  - File size: {file_size} bytes\n")
            else:
                print(f"Could not read image: {os.path.basename(file_path)}")
        except Exception as e:
            print(f"Error processing {os.path.basename(file_path)}: {e}")

# run the function to check 10 random images from Train and Test directories
check_random_image_sizes(Train_images, num_samples=10)
check_random_image_sizes(Test_images, num_samples=10)

Image: Scott_Oriole_0018_795840.jpg
  - Dimensions: 142 x 160
  - Channels: 3
  - File size: 9885 bytes

Image: Cedar_Waxwing_0075_179114.jpg
  - Dimensions: 388 x 500
  - Channels: 3
  - File size: 183677 bytes

Image: Brewer_Sparrow_0052_107478.jpg
  - Dimensions: 333 x 500
  - Channels: 3
  - File size: 127367 bytes

Image: Black_Throated_Blue_Warbler_0061_161667.jpg
  - Dimensions: 500 x 349
  - Channels: 3
  - File size: 122929 bytes

Image: Kentucky_Warbler_0066_165290.jpg
  - Dimensions: 500 x 350
  - Channels: 3
  - File size: 102009 bytes

Image: Northern_Fulmar_0052_43857.jpg
  - Dimensions: 500 x 333
  - Channels: 3
  - File size: 46576 bytes

Image: Sage_Thrasher_0093_155501.jpg
  - Dimensions: 500 x 333
  - Channels: 3
  - File size: 136660 bytes

Image: Purple_Finch_0006_27950.jpg
  - Dimensions: 500 x 500
  - Channels: 3
  - File size: 133371 bytes

Image: Yellow_Bellied_Flycatcher_0062_42693.jpg
  - Dimensions: 403 x 500
  - Channels: 3
  - File size: 141646 bytes

Imag

In [6]:
# base this off the model u use
IMG_SIZE = 300

SIZE = (IMG_SIZE, IMG_SIZE)
# base this off the nu,ber of classification species
NUM_CLASSES = 200

# now we make a validation set too, using image generator
data_gen = ImageDataGenerator(
    rescale = 1./255,
    # we want to make a validation set too, make it abt 20% of training data
    validation_split = .2,
    # this parameter, to my understanding, is to help the model better view distorted images / make the model more robust
    shear_range = .2,
    zoom_range = .2,
    horizontal_flip = True
)


In [7]:
# lets apply the data generator to the train set
train_data = data_gen.flow_from_directory(
    # remember we have the file paths defined above so we use them here
    Train_images,
    target_size = SIZE,
    color_mode='rgb',
    # from what i understand, this will be the amount of training cycle iterations
    batch_size = 32,
    class_mode = 'categorical',
    subset = 'training',
    # pictures shown in random order
    shuffle = True,
    # seed is 42 to keep consistency
    seed = 42
)

val_data = data_gen.flow_from_directory(
    # remember we have the file paths defined above so we use them here
    Train_images,
    target_size = SIZE,
    color_mode='rgb',
    # from what i understand, this will be the amount of training cycle iterations
    batch_size = 32,
    class_mode = 'categorical',
    # only real change between this and train data above
    subset = 'validation',
    # pictures shown in random order
    shuffle = True,
    # seed is 42 to keep consistency
    seed = 42
)
# for test set, we keep it simple, only resize it
test_data = ImageDataGenerator(
    rescale = 1./255
)

# now let's load in the test data with the datagen specifictions
test_data = test_data.flow_from_directory(
    Test_images,
    target_size = SIZE,
    batch_size = 32,
    class_mode = 'categorical',
    color_mode = 'rgb',
    # keep it in order for test images
    shuffle = False
)

Found 4869 images belonging to 200 classes.
Found 1125 images belonging to 200 classes.
Found 5794 images belonging to 200 classes.


In [10]:
# now lets built the basic model
base_model = EfficientNetB3(
    input_shape = (IMG_SIZE, IMG_SIZE, 3),
    # we choose False when we want to fine tune more intensly
    include_top = False,
    weights = 'imagenet'
)

#we're adding custom layers on top of model
model = models.Sequential([
    base_model,
    layers.GlobalAveragePooling2D(),
    layers.Dense(128, activation = 'relu'),
    # this should help with overfitting
    layers.Dropout(.5),
    layers.Dense(NUM_CLASSES, activation = 'softmax')
])

# now lets compile
model.compile(
    optimizer = optimizers.Adam(0.0001),
    loss = 'categorical_crossentropy',
    metrics = ['accuracy']
)

print(model.summary())

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ efficientnetb3 (Functional)     │ (None, 10, 10, 1536)   │    10,783,535 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_1      │ (None, 1536)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 128)            │       196,736 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 200)            │        25,800 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,006,071 (41.98 MB)

 Trainable params: 10,918,768 (41.65 MB)

 Non-trainable params: 87,303 (341.03 KB)

None


In [ ]:
# now we train omg
history = model.fit(
    train_data,
    # i dont think we neeed this: steps_per_epoch = len(train_images),
    # will iterate over training data ten times
    epochs = 10,
    #model uses it to test performance each time
    validation_data = val_data,
    callbacks = [
        # will stop model if it stops improving, prevents overfitting
        EarlyStopping(monitor = 'val_loss',
                      # waits 5 epochs before stopping
                      patience = 5,
                      # after stopping it gets best parameters
                      restore_best_weights = True),
        # redices learning rate when validation stops
        ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=2, mode='min')
    ]
)

Epoch 1/10
 42/153 ━━━━━━━━━━━━━━━━━━━━ 13:24 7s/step - accuracy: 0.0063 - loss: 5.3558

In [ ]:
''' 
Adjust the Learning Rate:
set initial learning rate lower, e.g., 0.0001, and let ReduceLROnPlateau adjust as necessary.

Class Weights:
Calculate class weights and pass them to model.fit() to handle class imbalance.

Gradual Fine-Tuning:
Freeze the base model initially and train only the added dense layers. Gradually unfreeze layers of the base model in steps.

Increase Training Data:
Use extensive data augmentation, or if possible, collect more data to bolster model training.
'''